# FP-growth
The FP-growth algorithm is described in the paper Han et al., Mining frequent patterns without candidate generation, where “FP” stands for frequent pattern. Given a dataset of transactions, the first step of FP-growth is to calculate item frequencies and identify frequent items. Different from Apriori-like algorithms designed for the same purpose, the second step of FP-growth uses a suffix tree (FP-tree) structure to encode transactions without generating candidate sets explicitly, which are usually expensive to generate. After the second step, the frequent itemsets can be extracted from the FP-tree. In MLlib, we implemented a parallel version of FP-growth called PFP, as described in Li et al., PFP: Parallel FP-growth for query recommendation. PFP distributes the work of growing FP-trees based on the suffices of transactions, and hence more scalable than a single-machine implementation. We refer users to the papers for more details.  

MLlib’s FP-growth implementation takes the following (hyper-)parameters:

* minSupport: the minimum support for an itemset to be identified as frequent. For example, if an item appears 3 out of 5 transactions, it has a support of 3/5=0.6.
* numPartitions: the number of partitions used to distribute the work.  

## Examples
FPGrowth implements the FP-growth algorithm. It take a RDD of transactions, where each transaction is an Array of items of a generic type. Calling FPGrowth.run with transactions returns an FPGrowthModel that stores the frequent itemsets with their frequencies. The following example illustrates how to mine frequent itemsets and association rules (see Association Rules for details) from transactions.

In [3]:
val PATH = "file:///Users/lzz/work/SparkML/"
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.fpm.FPGrowth

val data = sc.textFile(PATH+"data/mllib/sample_fpgrowth.txt")

val transactions: RDD[Array[String]] = data.map(s => s.trim.split(' '))

val fpg = new FPGrowth().setMinSupport(0.2).setNumPartitions(10)
val model = fpg.run(transactions)

model.freqItemsets.collect().foreach { itemset =>
  println(itemset.items.mkString("[", ",", "]") + ", " + itemset.freq)
}

val minConfidence = 0.8
model.generateAssociationRules(minConfidence).collect().foreach { rule =>
  println(
    rule.antecedent.mkString("[", ",", "]")
      + " => " + rule.consequent .mkString("[", ",", "]")
      + ", " + rule.confidence)
}

[z], 5
[x], 4
[x,z], 3
[y], 3
[y,x], 3
[y,x,z], 3
[y,z], 3
[r], 3
[r,x], 2
[r,z], 2
[s], 3
[s,y], 2
[s,y,x], 2
[s,y,x,z], 2
[s,y,z], 2
[s,x], 3
[s,x,z], 2
[s,z], 2
[t], 3
[t,y], 3
[t,y,x], 3
[t,y,x,z], 3
[t,y,z], 3
[t,s], 2
[t,s,y], 2
[t,s,y,x], 2
[t,s,y,x,z], 2
[t,s,y,z], 2
[t,s,x], 2
[t,s,x,z], 2
[t,s,z], 2
[t,x], 3
[t,x,z], 3
[t,z], 3
[p], 2
[p,r], 2
[p,r,z], 2
[p,z], 2
[q], 2
[q,y], 2
[q,y,x], 2
[q,y,x,z], 2
[q,y,z], 2
[q,t], 2
[q,t,y], 2
[q,t,y,x], 2
[q,t,y,x,z], 2
[q,t,y,z], 2
[q,t,x], 2
[q,t,x,z], 2
[q,t,z], 2
[q,x], 2
[q,x,z], 2
[q,z], 2
[t,s,y] => [x], 1.0
[t,s,y] => [z], 1.0
[y,x,z] => [t], 1.0
[y] => [x], 1.0
[y] => [z], 1.0
[y] => [t], 1.0
[p] => [r], 1.0
[p] => [z], 1.0
[q,t,z] => [y], 1.0
[q,t,z] => [x], 1.0
[q,y] => [x], 1.0
[q,y] => [z], 1.0
[q,y] => [t], 1.0
[t,s,x] => [y], 1.0
[t,s,x] => [z], 1.0
[q,t,y,z] => [x], 1.0
[q,t,x,z] => [y], 1.0
[q,x] => [y], 1.0
[q,x] => [t], 1.0
[q,x] => [z], 1.0
[t,x,z] => [y], 1.0
[x,z] => [y], 1.0
[x,z] => [t], 1.0
[p,z] => [r], 1.0
[t